## 모델 파인튜닝

### 데이터셋

In [ ]:
!pwd

In [ ]:
!ls -la ../input/finetuned_model/pytorch/default/1/yi-ko-6b-text2sql

In [ ]:
# !cp -rf ../input/finetuned_model/pytorch/default/1/yi-ko-6b-text2sql/* ./yi-ko-6b-text2sql

In [ ]:
!ls -la ./yi-ko-6b-text2sql

In [ ]:
!ls -la ../input/test-train-datasets/datasets

In [ ]:
!mkdir datasets

In [ ]:
cp ../input/test-train-datasets/datasets/train.csv ./datasets

### 기존 모델을 새로운 데이터셋으로 튜닝

In [ ]:
!pip install autotrain-advanced==0.7.77 -qqq

In [ ]:
base_model = 'beomi/Yi-Ko-6B'  # huggingface에 있는 원본 모델
finetuned_model = 'yi-ko-6b-text2sql'  # 교육 후 모델

# 셋팅 옵션 리스트
# --model {base_model} \
# --project-name {finetuned_model} \
# --data-path
# --text-column text \

# --use-peft 양자화 옵션
!autotrain llm \
--train \
--model {base_model} \
--project-name {finetuned_model} \
--data-path datasets/ \
--text-column prompt \
--lr 2e-4 \
--batch-size 2 \
--epochs 1 \
--block-size 1024 \
--warmup-ratio 0.1 \
--weight-decay 0.01 \
--gradient-accumulation 8 \
--mixed-precision fp16 \
--use-peft \
--lora-r 16 \
--lora-alpha 32 \
--lora-dropout 0.05 \
--quantization int4 \
--trainer sft

In [ ]:
# model_name = base_model


base_model = 'beomi/Yi-Ko-6B'
finetuned_model = 'yi-ko-6b-text2sql'

# device_map = {"": 0}
device_map = "auto"

## 기존 참조 모델과 튜닝 모델 머지

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, PeftModel
import os
from huggingface_hub import login

# Hugging Face 토큰으로 로그인

os.environ['HF_TOKEN'] = ""



# LoRA와 기초 모델 파라미터 합치기
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, finetuned_model)
model = model.merge_and_unload()

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_size = 'right'

In [ ]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['HF_TOKEN'] = user_secrets.get_secret("HF_TOKEN")
os.environ['GROQ_API_KEY'] = user_secrets.get_secret("GROQ_API_KEY")

In [ ]:
# Finetuning된 모델 Hugging Face 등록

model.push_to_hub(finetuned_model, use_temp_dir=False)

In [ ]:
# 사용될(Finetuned Model 맞는) Tokenizer를 Hugging Face 등록

tokenizer.push_to_hub(finetuned_model, use_temp_dir=False)